# Title

This Notebook...

## Initialization

### Properties

In [ ]:
date = "20171120"
roadNumber = 2
roadsFileName = "Test_Data\BPS_20171120.txt"
detectionsFileName = "Test_Data\A2_20171120.txt"
outputDirectory = "Test_Data"

### Imports

In [ ]:
import sys
import datetime
import csv
import time as timeModule
import numpy
import scipy
import scipy.ndimage
import scipy.ndimage.filters
import matplotlib.pyplot
import matplotlib.patches

### Helper Functions

In [ ]:
debugMode = True;

In [ ]:
def dprint(objectToPrint, depth = 0):
    indent = "  " * depth
    if (debugMode):
        print(indent + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + " | " + str(objectToPrint))

In [ ]:
def progressBar(message, value, endvalue, bar_length = 20):
    if debugMode:
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        sys.stdout.write("\r" 
                         + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
                         + " | " 
                         + message 
                         + ": [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        if value == endvalue:
            sys.stdout.write("\n")
        sys.stdout.flush()

In [ ]:
def extractAttribute(bpsCode, startBit, endBit):
    return int(bpsCode, 16) >> (80 - endBit) & int("1" * (endBit - startBit), 2)

In [ ]:
print(extractAttribute("00D00C03405B18200005", 14, 24))
print(extractAttribute("00D00C03405B18200005", 24, 38))

## Classes

In [ ]:
class BPSDetector:
    bpsCode = None
    roadNumber = None
    hectometer = None
    additionalMeters = None
    def __init__(self, bpsCode):
        self.bpsCode = bpsCode
        self.roadNumber = extractAttribute(bpsCode, 14, 24)
        self.hectometer = extractAttribute(bpsCode, 24, 38)
        self.additionalMeters = extractAttribute(bpsCode, 38, 48)
    def getBPSCoder(self):
        return self.bpsCode
    def getRoadNumber(self):
        return self.roadNumber
    def getHectometer(self):
        return self.hectometer
    def getAdditionalMeters(self):
        return self.additionalMeters
    def getMeter(self):
        return 100 * self.hectometer + self.additionalMeters
    def __str__(self):
        return "bpsCode: {} | roadNumber: {} | hectometer: {} | additionalMeters: {}".format(self.bpsCode, self.roadNumber, self.hectometer, self.additionalMeters)

In [ ]:
bpsDetector = BPSDetector("00D00C03405B18200005")
print(bpsDetector)

In [ ]:
class Road:
    roadNumber = None
    bpsDetectors = None
    spaceToSpaceIndex = None
    def __init__(self, roadNumber):
        self.roadNumber = roadNumber
        self.bpsDetectors = set()
        self.spaceToSpaceIndex = dict()
    def addBPSDetector(self, bpsDetector):
        self.bpsDetectors.add(bpsDetector)
    def indexDetectorSpaces(self):
        spaceSet = set()
        for bpsDetector in self.bpsDetectors:
            spaceSet.add(bpsDetector.getMeter())
        sortedSpaceSet = sorted(spaceSet)
        for spaceIndex in range(len(sortedSpaceSet)):
            self.spaceToSpaceIndex[sortedSpaceSet[spaceIndex]] = spaceIndex
    def getBPSDetectors(self):
        return self.bpsDetectors
    def getSpaceToSpaceIndex(self):
        return self.spaceToSpaceIndex
    def __str__(self):
        return "roadNumber: {} | len(bpsDetectors): {} | len(spaceToSpaceIndex): {}".format(self.roadNumber, len(self.bpsDetectors), len(self.spaceToSpaceIndex))

In [ ]:
road = Road(12)
road.addBPSDetector(bpsDetector)
print(road)
road.indexDetectorSpaces()
print(road.getSpaceToSpaceIndex())

In [ ]:
class Detection:
    code = None
    space = None
    time = None
    speed = None
    flow = None
    def __init__(self, code, space, time, speed, flow):
        self.code = code
        self.space = space
        self.time = time
        self.speed = speed
        self.flow = flow
    def getCode(self):
        return self.code
    def getSpace(self):
        return self.space
    def getTime(self):
        return self.time
    def getSpeed(self):
        return self.speed
    def getFlow(self):
        return self.flow
    def __str__(self):
        return "code: {} | space: {} | time: {} | speed: {} | flow: {}".format(self.code, self.space, self.time, self.speed, self.flow)

In [ ]:
detection = Detection("00D00C03405B18200005", 1234, 1440, 120, 17)
print(detection)

## Roads

In [ ]:
def readCSVToBPSCodes(fileName):
    dprint("Starting readCSVToBPSCodes()")
    result = []
    with open(fileName, "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[3] == "R":
                result.append(row[0])
    dprint("Ending readCSVToBPSCodes()")
    return result

In [ ]:
bpsCodes = readCSVToBPSCodes(roadsFileName)

In [ ]:
for bpsCode in bpsCodes:
    print(bpsCode)

In [ ]:
def parseBPSCodesToRoads(bpsCodes):
    dprint("Starting parseBPSCodesToRoads()")
    result = dict()
    for bpsCode in bpsCodes:
        bpsDetector = BPSDetector(bpsCode)
        roadNumber = bpsDetector.getRoadNumber()
        if roadNumber not in result:
            road = Road(roadNumber)
            result[roadNumber] = road
        result[roadNumber].addBPSDetector(bpsDetector)
    for key, value in result.items():
        value.indexDetectorSpaces()
    dprint("Ending parseBPSCodesToRoads()")
    return result

In [ ]:
roads = parseBPSCodesToRoads(bpsCodes)

In [ ]:
for value in sorted(roads.values(), key = lambda x:len(x.getBPSDetectors())):
    print(value)

## Speed and Flows

In [ ]:
def readCSVToDetections(fileName):
    dprint("Starting readCSVToDetections()")
    result = set()
    with open(fileName, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter = " ")
        next(reader, None)
        next(reader, None)
        next(reader, None)
        for row in reader:
            if len(row) > 1 and row[2] == "R-":
                code = row[0]
                space = int(row[1][:-1])
                timeObject = timeModule.strptime(row[6], "%H:%M")
                time = 60 * timeObject.tm_hour + timeObject.tm_min
                speed = int(row[9])
                flow = float(row[8]) / float(row[4])
                detection = Detection(code, space, time, speed, flow)
                result.add(detection)
    dprint("Ending readCSVToDetections()")
    return result

In [ ]:
detections = readCSVToDetections(detectionsFileName)

In [ ]:
iterator = iter(detections)
for i in range(min(len(detections), 300)):
    detection = next(iterator)
    print(detection)

In [ ]:
def parseDetectionsToSpeedFlows(detections, road):
    dprint("Starting parseDetectionsToSpeedFlows()")
    spaceToSpaceIndex = road.getSpaceToSpaceIndex()
    maxSpaceIndex = max(spaceToSpaceIndex.values()) + 1
    maxTimeIndex = 1440 # Trivial now, but perhaps important later.
    speeds = numpy.full((maxSpaceIndex, maxTimeIndex), numpy.nan, dtype = numpy.int32)
    flows = numpy.full((maxSpaceIndex, maxTimeIndex), numpy.nan, dtype = numpy.int32)
    minFoundSpaceIndex = maxSpaceIndex
    maxFoundSpaceIndex = 0
    minFoundTimeIndex = maxTimeIndex
    maxFoundTimeIndex = 0
    for detection in detections:
        space = detection.getSpace()
        time = detection.getTime()
        speed = detection.getSpeed()
        flow = detection.getFlow()
        spaceIndex = spaceToSpaceIndex[space]
        timeIndex = time # Trivial now, but perhaps important later.
        if spaceIndex < minFoundSpaceIndex:
            minFoundSpaceIndex = spaceIndex
        if spaceIndex > maxFoundSpaceIndex:
            maxFoundSpaceIndex = spaceIndex
        if timeIndex < minFoundTimeIndex:
            minFoundTimeIndex = timeIndex
        if timeIndex > maxFoundTimeIndex:
            maxFoundTimeIndex = timeIndex
        speeds[spaceIndex, timeIndex] = speed
        flows[spaceIndex, timeIndex] = flow
    speeds = speeds[minFoundSpaceIndex:maxFoundSpaceIndex + 1, minFoundTimeIndex:maxFoundTimeIndex + 1]
    flows = flows[minFoundSpaceIndex:maxFoundSpaceIndex + 1, minFoundTimeIndex:maxFoundTimeIndex + 1]
    dprint("Ending parseDetectionsToSpeedFlows()")
    return speeds, flows, minFoundSpaceIndex, maxFoundSpaceIndex, minFoundTimeIndex, maxFoundTimeIndex

In [ ]:
speeds, flows, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex = parseDetectionsToSpeedFlows(detections, roads[roadNumber])

In [ ]:
print(speeds.shape)
print(flows.shape)
print(minSpaceIndex)
print(maxSpaceIndex)
print(minTimeIndex)
print(maxTimeIndex)
matplotlib.pyplot.imshow(speeds, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flows, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
def removeLowFlowTimes(speeds, flows):
    dprint("Starting removeLowFlowTimes()")
    mask = numpy.nanmean(flows, axis = 0) > 10
    speeds = speeds[:,mask]
    flows = flows[:,mask]
    dprint("Ending removeLowFlowTimes()")
    return speeds, flows, mask

In [ ]:
speedsHighFlow, flowsHighFlow, maskHighFlow = removeLowFlowTimes(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsHighFlow.shape)
print(flowsHighFlow.shape)
print(sum(maskHighFlow) / (sum(maskHighFlow) + sum(~maskHighFlow)))
matplotlib.pyplot.imshow(speedsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
def removeMissingDetectors(speeds, flows):
    dprint("Starting removeMissingDetectors()")
    mask = ~numpy.isnan(speeds).all(axis = 1)
    speeds = speeds[mask]
    flows = flows[mask]
    dprint("Ending removeMissingDetectors()")
    return speeds, flows, mask

In [ ]:
speedsWorkingDetectors, flowsWorkingDetectors, maskWorkingDetectors = removeMissingDetectors(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsWorkingDetectors.shape)
print(flowsWorkingDetectors.shape)
print(sum(maskWorkingDetectors) / (sum(maskWorkingDetectors) + sum(~maskWorkingDetectors)))
matplotlib.pyplot.imshow(speedsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion

In [ ]:
def parseSpeedFlowsToCongestions(speeds, flows):
    dprint("Starting parseSpeedFlowsToCongestions()")
    congestions = speeds / 65# + flows / 40
    dprint("Ending parseSpeedFlowsToCongestions()")
    return congestions

In [ ]:
congestions = parseSpeedFlowsToCongestions(speedsWorkingDetectors, flowsWorkingDetectors)

In [ ]:
print(congestions.shape)
matplotlib.pyplot.imshow(congestions, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
def interpolateMissingValues(congestions):
    dprint("Starting interpolateMissingValues()")
    x = numpy.arange(0, congestions.shape[1])
    y = numpy.arange(0, congestions.shape[0])
    congestionsMask = numpy.ma.masked_invalid(congestions)
    xx, yy = numpy.meshgrid(x, y)
    x1 = xx[~congestionsMask.mask]
    y1 = yy[~congestionsMask.mask]
    newarr = congestions[~congestionsMask.mask]
    congestions = scipy.interpolate.griddata((x1, y1), newarr.ravel(), (xx, yy), method = "cubic")
    dprint("Ending interpolateMissingValues()")
    return congestions

In [ ]:
congestionsWithoutMissingValues = interpolateMissingValues(congestions)

In [ ]:
print(congestionsWithoutMissingValues.shape)
matplotlib.pyplot.imshow(congestionsWithoutMissingValues, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
def applySmoothingFilter(congestions):
    dprint("Starting applySmoothingFilter()")
    #congestions = scipy.ndimage.filters.gaussian_filter(congestions, 5)
    congestions = scipy.ndimage.filters.uniform_filter(congestions, [10, 20])
    dprint("Ending applySmoothingFilter()")
    return congestions

In [ ]:
congestionsSmoothed = applySmoothingFilter(congestionsWithoutMissingValues)

In [ ]:
print(congestionsSmoothed.shape)
matplotlib.pyplot.imshow(congestionsSmoothed, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion Filter

In [ ]:
testDataArray = numpy.random.rand(16,9)
print(testDataArray >= 0.9)

In [ ]:
def addBoundaries(ax, boundaries):
    rect = matplotlib.patches.Rectangle((
        boundaries[2] - 0.5,
        boundaries[0] - 0.5),
        boundaries[3] - boundaries[2] + 1,
        boundaries[1] - boundaries[0] + 1,
        linewidth = 1,
        edgecolor = "r",
        hatch = "//",
        facecolor = "none")
    ax.add_patch(rect)

In [ ]:
def showDataWithBoundaries(dataArray, boundariesList):
    fig, ax = matplotlib.pyplot.subplots(1)
    ax.imshow(dataArray, aspect = "auto")
    for i in range(len(boundariesList)):
        addBoundaries(ax, boundariesList[i])
    matplotlib.pyplot.show()

In [ ]:
def scanForBoundaries(dataArray, boundaries = None, threshold = 1, depth = 0):
    dprint("Starting scanForBoundaries()", depth)
    if boundaries is None:
        boundaries = [0, dataArray.shape[0] - 1, 0, dataArray.shape[1] - 1]
    boundariesList = []
    subArray = dataArray[boundaries[0]:boundaries[1] + 1, boundaries[2]:boundaries[3] + 1]
    arrayBoolean = numpy.any(subArray < threshold, depth % 2)
    if depth % 2:
        length = boundaries[1] + 1 - boundaries[0]
    else:
        length = boundaries[3] + 1 - boundaries[2]
    started = False
    if length > 1:
        childBoundaries = boundaries[:]
        for i in range(length):
            if arrayBoolean[i] and not started:
                dprint("Found start: " + str(i), depth)
                if depth % 2:
                    childBoundaries[0] = boundaries[0] + i
                else:
                    childBoundaries[2] = boundaries[2] + i
                started = True
            elif not arrayBoolean[i] and started:
                dprint("Found stop: " + str(i - 1), depth)
                if depth % 2:
                    childBoundaries[1] = boundaries[0] + i - 1
                else:
                    childBoundaries[3] = boundaries[2] + i - 1
                started = False
                boundariesList.append(childBoundaries[:])
        if started:
            dprint("Found stop at end.", depth)
            if depth % 2:
                childBoundaries[1] = boundaries[1]
            else:
                childBoundaries[3] = boundaries[3]
            boundariesList.append(childBoundaries)
    else:
        dprint("Length is one", depth)
        boundariesList.append(boundaries)
    dprint("Ending scanForBoundaries()", depth)
    return boundariesList

In [ ]:
scanForBoundaries(testDataArray, threshold = 0.1, depth = 0)

In [ ]:
def recursiveScanForBoundaries(dataArray, boundaries = None, depth = 0, parentLength = 0, threshold = 1): # Remove copies?
    dprint("Starting recursiveScanForBoundaries()", depth)
    dprint("Direction: " + str(depth % 2), depth)
    result = []
    boundariesList = scanForBoundaries(dataArray, boundaries, threshold, depth)
    if debugMode:
        showDataWithBoundaries(dataArray, boundariesList)
    length = len(boundariesList)
    dprint("length: " + str(length) + "| parentLength: " + str(parentLength), depth)
    if length == 0:
        dprint("Error: nothing found...", depth) # Throw Exception?
    elif length == 1 and parentLength == 1:
        dprint("Done: " + str(boundariesList[0]), depth)
        return boundariesList
    else:
        dprint("Scanning children...", depth)
        parentLength = length;
        childDepth = depth + 1;
        for i in range(length):
            dprint("Child: " + str(i), depth)
            childBoundaries = boundariesList[i][:]
            result += recursiveScanForBoundaries(dataArray, childBoundaries, childDepth, parentLength, threshold)
    dprint("Ending recursiveScanForBoundaries()", depth)
    return result

In [ ]:
testBoundariesList = recursiveScanForBoundaries(testDataArray, threshold = 0.1)

In [ ]:
testBoundariesList

In [ ]:
showDataWithBoundaries(testDataArray, testBoundariesList)
showDataWithBoundaries(testDataArray < 0.1, testBoundariesList)

In [ ]:
debugMode = False;

In [ ]:
congestionBoundariesList = recursiveScanForBoundaries(congestionsSmoothed)

In [ ]:
print(congestionBoundariesList)

In [ ]:
def filterLargeCongestions(congestionBoundariesList):
    result = []
    for congestionBoundaries in congestionBoundariesList:
        size = (congestionBoundaries[1] - congestionBoundaries[0]) * (congestionBoundaries[3] - congestionBoundaries[2])
        if size > 1000:
            result.append(congestionBoundaries)
    return result

In [ ]:
congestionBoundariesListFiltered = filterLargeCongestions(congestionBoundariesList)

In [ ]:
print(congestionBoundariesListFiltered)

In [ ]:
def addMargins(congestionBoundariesList): # TODO: implement
    result = []
    marginSpace = 10
    marginTime = 20
    for congestionBoundaries in congestionBoundariesList:
        congestionBoundaries[0] = max(minSpaceIndex, congestionBoundaries[0] - marginSpace)
        congestionBoundaries[1] = min(maxSpaceIndex, congestionBoundaries[1] + marginSpace)
        congestionBoundaries[2] = max(minTimeIndex, congestionBoundaries[2] - marginTime)
        congestionBoundaries[3] = min(maxTimeIndex, congestionBoundaries[3] + marginTime)
        result.append(congestionBoundaries)
    return result

In [ ]:
congestionBoundariesListWithMargins = addMargins(congestionBoundariesListFiltered)

In [ ]:
print(congestionBoundariesListWithMargins)

In [ ]:
showDataWithBoundaries(congestionsSmoothed, congestionBoundariesListWithMargins)
showDataWithBoundaries(congestionsSmoothed < 1, congestionBoundariesList)

In [ ]:
showDataWithBoundaries(speeds, congestionBoundariesListWithMargins)

In [ ]:
def writeSpeedsAndFlowsToCSV(speeds, flows, congestionBoundariesList, outputDirectory):
    for congestionBoundaries in congestionBoundariesList:
        minSpaceIndex = congestionBoundaries[0] # Index to Space
        maxSpaceIndex = congestionBoundaries[1]
        minTimeIndex = congestionBoundaries[2] # Index to Time
        maxTimeIndex = congestionBoundaries[3]
        speedsFileName = outputDirectory + "\\" + str(date) + "_" + str(roadNumber) + "_s_" + str(minSpaceIndex) + "-" + str(maxSpaceIndex) + "_" + str(minTimeIndex) + "-" + str(maxTimeIndex) + ".csv.gz"
        flowsFileName = outputDirectory + "\\" + str(date) + "_" + str(roadNumber) + "_f_" + str(minSpaceIndex) + "-" + str(maxSpaceIndex) + "_" + str(minTimeIndex) + "-" + str(maxTimeIndex) + ".csv.gz"
        boundedSpeeds = speeds[minSpaceIndex:maxSpaceIndex, minTimeIndex:maxTimeIndex]
        boundedFlows = flows[minSpaceIndex:maxSpaceIndex, minTimeIndex:maxTimeIndex]
        numpy.savetxt(speedsFileName, boundedSpeeds, fmt = "%d", delimiter = ",")
        numpy.savetxt(flowsFileName, boundedFlows, fmt = "%d", delimiter = ",")

In [ ]:
writeSpeedsAndFlowsToCSV(speeds, flows, congestionBoundariesListWithMargins, outputDirectory)

End of Notebook